In [2]:
import pandas as pd
import numpy as np
#load data
df = pd.read_csv("./csv/0_500_union_salida_clasificada.csv")

In [5]:
df.sample(5)

,Clase,Mensaje
466,Negativo,Con la novedad de que el cambio de la pieza da...
127,Negativo,@Aerolineas_AR Dejen de mentir!!!! después se ...
229,Neutro,"@vicky__r @fibertel Buenas tardes Vicky, mi no..."
436,Negativo,¿Alguien más está harto de la cantidad de cabl...
255,Negativo,@fibertel pésimo servicio. Hace una semana que...


In [7]:
from collections import Counter
Counter(df["Clase"])

Counter({'Positivo': 124, 'Neutro': 121, 'Negativo': 256})

In [10]:
X = []
for i in range(df.shape[0]):
    X.append(df.iloc[i][1])
y = np.array(df["Clase"])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

#pipeline of feature engineering and model
model = Pipeline([('vectorizer', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#the class_weight="balanced" option tries to remove the biasedness of model towards majority sample

In [30]:
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = [{
              'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
              'tfidf__use_idf': (True, False),
              'clf': (OneVsRestClassifier(LinearSVC(class_weight="balanced")),)
             },
             {
              'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
              'tfidf__use_idf': (True, False),
              'clf': (MLPClassifier(),),
              'clf__solver': ['lbfgs'], 
              'clf__max_iter': [500,1000,1500], 
              'clf__alpha': 10.0 ** -np.arange(1, 7), 
              'clf__hidden_layer_sizes':np.arange(5, 12), 
              'clf__random_state':[0,1,2,3,4,5,6,7,8,9] 
             }
            ]


gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

.conda/envs/orange3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
.conda/envs/orange3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.7305389221556886
{'clf': MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=5, learning_rate='constant',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=3, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False), 'clf__alpha': 0.1, 'clf__hidden_layer_sizes': 5, 'clf__max_iter': 500, 'clf__random_state': 3, 'clf__solver': 'lbfgs', 'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}
